In [48]:
from collections import Counter
import os
import re

import numpy as np
import pandas as pd

import pprint

In [49]:
# Helper functions

def isNaN(num):
    return num != num

In [50]:
# Read RoO data
roo_df = {}

roo_folder = 'RoO Table/roo_clean'
for filename in os.listdir(roo_folder):
    roo_df[filename[:-4]] = pd.read_csv(roo_folder + '/' + filename)

In [51]:
for key in roo_df:
    print(key)

roo_df['ASEAN_JPN'].head()
#roo_df['JPN_VNM'][roo_df['JPN_VNM'].iloc[:, 5].notnull()]

ASEAN_JPN
AUS_JPN
EU_JPN
JPN_MNG
JPN_VNM


,Tariff item number,Unnamed: 1,Unnamed: 2,Description of goods,Product specific rules
0,Section I Live animals; animal products (chapt...,NaN,NaN,NaN,NaN
1,Chapter 1,NaN,NaN,Live animals,CC
2,Chapter 2,NaN,NaN,Meat and edible meat offal,"CC, except from chapter 1"
3,Chapter 3,NaN,NaN,"Fish and crustaceans, molluscs and other Aquat...",CC
4,Chapter 4,NaN,NaN,Dairy produce; birds’ eggs; natural honey; edi...,CC


In [52]:
# Handle ASEAN_JPN and JPN_VNM (5 columns)
roo_rules = {}

df = roo_df['ASEAN_JPN']
df2 = df.loc[df.iloc[:, 4].notna()].iloc[:, [0, 1, 2, 4]]
roo_cont = {}
for row in df2.itertuples(index=False, name=None):
    # Fetch hs code
    if not isNaN(row[2]):
        hs_code = row[2]
    elif not isNaN(row[1]):
        hs_code = row[1]
    elif not isNaN(row[0]):
        hs_code = row[0]
    else:
        hs_code = None
    if hs_code:
        roo_cont[hs_code] = row[3]
roo_rules['ASEAN_JPN'] = roo_cont


df = roo_df['JPN_VNM']
df2 = df.loc[df.iloc[:, 4].notna()].iloc[:, [0, 1, 2, 4]]
roo_cont = {}
for row in df2.itertuples(index=False, name=None):
    # Fetch hs code
    if not isNaN(row[2]):
        hs_code = row[2]
    elif not isNaN(row[1]):
        hs_code = row[1]
    elif not isNaN(row[0]):
        hs_code = row[0]
    else:
        hs_code = None
    if hs_code:
        roo_cont[hs_code] = row[3]
roo_rules['JPN_VNM'] = roo_cont

In [53]:
# Handle AUS_JPN (4 columns)
# Several rules are (num), falsely converted into -num instead
df = roo_df['AUS_JPN']
df2 = df.loc[df.iloc[:, 2].notna()].iloc[:, [0, 2, 3]]
roo_cont = {}
for row in df2.itertuples(index=False, name=None):
    # Fetch hs code
    if not isNaN(row[0]):
        hs_code = row[0]
    else:
        hs_code = None
    if hs_code:
        roo_cont[hs_code] = row[1]
roo_rules['AUS_JPN'] = roo_cont

In [54]:
# Handle EU_JPN (2 columns)
# Need to be handled differently
df = roo_df['EU_JPN']
df2 = df.loc[df.iloc[:, 1].notna()]
roo_cont = {}
for row in df2.itertuples(index=False, name=None):
    # Fetch hs code
    if not isNaN(row[0]):
        hs_code = row[0]
    else:
        hs_code = None
    if hs_code:
        roo_cont[hs_code] = row[1]
roo_rules['EU_JPN'] = roo_cont

In [55]:
# Handle JPN_MNG (4 columns)
df = roo_df['JPN_MNG']
df2 = df.loc[df.iloc[:, 1].notna()]
roo_cont = {}
for row in df2.itertuples(index=False, name=None):
    # Fetch hs code
    if not isNaN(row[2]):
        hs_code = row[2]
    elif not isNaN(row[1]):
        hs_code = row[1]
    elif not isNaN(row[0]):
        hs_code = row[0]
    else:
        hs_code = None
    if hs_code:
        roo_cont[hs_code] = row[3]
roo_rules['JPN_MNG'] = roo_cont

In [56]:
# Count types of RoO in every FTA
roo_types_count = {}

for fta in roo_rules:
    freq = Counter()
    for hs_code, rule in roo_rules[fta].items():
        # Chapter level
        if 'CC' in rule:
            freq['chapter'] += 1
        
        # Heading level
        elif 'CTH' in rule:
            freq['heading'] += 1
        
        # Subheading level
        # Note: There are a few specific rules related to chemicals (e.g. alkali metal; "or any other subheading")
        elif 'CTSH' in rule:
            freq['subheading'] += 1
            
        # "Wholly obtained"
        elif 'WO' in rule:
            freq['wholly'] += 1

        # Manufacture criteria?
        elif 'Manufacture from' in rule:
            freq['manufacture'] += 1
        
        # Value added?
        elif 'RVC' in rule or 'LVC' in rule:
            freq['value'] += 1
            
        # Special for AUS_JPN
        elif rule in map(str, range(-17, 1)):
            freq['special'] += 1
            
        # No rules imposed
        elif 'No required change in tariff' in rule or 'No change in tariff' in rule:
            freq['no rule'] += 1
            
        # Other notes
        else:
            freq['else'] += 1
            # Catch for error in data
            #print(hs_code)
            #print(rule)
            #print()
            
    roo_types_count[fta] = freq
    print(fta)
    pprint.pprint(freq, width=35)

ASEAN_JPN
Counter({'chapter': 286,
         'value': 92,
         'heading': 84,
         'subheading': 30,
         'wholly': 2,
         'else': 1})
JPN_VNM
Counter({'subheading': 638,
         'chapter': 365,
         'heading': 96,
         'wholly': 37,
         'value': 11,
         'else': 2})
AUS_JPN
Counter({'subheading': 865,
         'heading': 816,
         'chapter': 239,
         'special': 224,
         'else': 1})
EU_JPN
Counter({'else': 190,
         'heading': 185,
         'subheading': 62,
         'chapter': 28,
         'value': 6,
         'wholly': 4})
JPN_MNG
Counter({'heading': 406,
         'subheading': 263,
         'chapter': 119,
         'manufacture': 50,
         'wholly': 15})


In [57]:
print('FTA\t\tChapter\t\tHeading\t\tSubheading\tOther\t\tTotal')
for fta, rules in roo_rules.items():
    total = len(rules)
    chapter = roo_types_count[fta]['chapter'] / total
    heading = roo_types_count[fta]['heading'] / total
    subheading = roo_types_count[fta]['subheading'] / total
    other = 1 - (chapter + heading + subheading)
    if fta == 'ASEAN_JPN':
        print('{}\t{:2.2%}\t\t{:2.2%}\t\t{:2.2%}\t\t{:2.2%}\t\t{}'.format(fta, chapter, heading, subheading, other, total))
        continue
    print('{}\t\t{:2.2%}\t\t{:2.2%}\t\t{:2.2%}\t\t{:2.2%}\t\t{}'.format(fta, chapter, heading, subheading, other, total))

FTA		Chapter		Heading		Subheading	Other		Total
ASEAN_JPN	57.78%		16.97%		6.06%		19.19%		495
JPN_VNM		31.77%		8.36%		55.53%		4.35%		1149
AUS_JPN		11.14%		38.04%		40.33%		10.49%		2145
EU_JPN		5.89%		38.95%		13.05%		42.11%		475
JPN_MNG		13.95%		47.60%		30.83%		7.62%		853
